In [518]:
#*****************************************************************************
#**
#**  Pandas Homework: Pandas, Pandas, Pandas
#**
#**   Heroes of Pymoli
#**
#**     Author: George Alonzo
#**   Due Date: Sept 25, 2021
#**
#*****************************************************************************

import pandas as pd
import numpy as np

purchase_file = "Resources/purchase_data.csv"

purchase_file_df = pd.read_csv(purchase_file)
#purchase_file_df["Price"] = purchase_file_df["Price"].map("${:.2f}".format)
purchase_file_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [519]:
# PLAYER COUNT
print("*** THIS IS OK *** PLAYER COUNT")

player_num = purchase_file_df["SN"].count()
player_num_unique = len(purchase_file_df["SN"].unique())

player_count_df = pd.DataFrame({"Total Players (Unique)": [player_num_unique]})
player_count_df

*** THIS IS OK *** PLAYER COUNT


,Total Players (Unique)
0,576


In [520]:
# PURCHASING ANALYSIS (TOTAL)
print("*** THIS IS OK *** PURCHASING ANALYSIS (TOTAL)")

item_unique = len(purchase_file_df["Item Name"].unique())
avg_price = purchase_file_df["Price"].mean()
purchases_num = purchase_file_df["Purchase ID"].count()
tot_revenue = purchase_file_df["Price"].sum()

purchasing_analysis_df = pd.DataFrame({"Number of Unique Items": [item_unique], "Average Price": [avg_price],
                                       "Number of Purchases": [purchases_num], "Total Revenue": [tot_revenue]})
purchasing_analysis_df["Average Price"] = purchasing_analysis_df["Average Price"].map("${:.2f}".format)
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].map("${:,.2f}".format)

purchasing_analysis_df

*** THIS IS OK *** PURCHASING ANALYSIS (TOTAL)


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [521]:
# GENDER DEMOGRAPHICS
print("*** THIS IS OK *** GENDER DEMOGRAPHICS")

#Create new DF with distinct Screen Names for demographics
gender_distinct_df = purchase_file_df[["Gender","SN"]].drop_duplicates()


distinct_tot = gender_distinct_df["SN"].count()
gender_counts = gender_distinct_df["Gender"].value_counts()
gender_pcts = (gender_distinct_df["Gender"].value_counts() / distinct_tot) *100
gender_demo_df = pd.DataFrame({"Total Count": gender_counts, "Percentage": gender_pcts})
gender_demo_df["Percentage"] = gender_demo_df["Percentage"].map("{:.2f}%".format)
gender_demo_df

*** THIS IS OK *** GENDER DEMOGRAPHICS


,Total Count,Percentage
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [528]:
# PURCHASING ANALYSIS (GENDER)
print("*** THIS IS OK *** PURCHASING ANALYSIS (GENDER)")

purchases_df = purchase_file_df[["Gender","Price"]]
purchases_df.head()

purchases_group_df = purchases_df.groupby(["Gender"]).count()
purchases_group_df.columns = ["Purchase Count"]
purchases_group_df["Average Purchase Price"] = purchases_df.groupby(["Gender"]).mean()
purchases_group_df["Total Purchase Price"] = purchases_df.groupby(["Gender"]).sum()
purchases_group_df["Avg Total Purchase Price"] = purchases_group_df["Total Purchase Price"] / gender_counts

purchases_group_df["Average Purchase Price"] = purchases_group_df["Average Purchase Price"].map("${:,.2f}".format)
purchases_group_df["Total Purchase Price"] = purchases_group_df["Total Purchase Price"].map("${:,.2f}".format)
purchases_group_df["Avg Total Purchase Price"] = purchases_group_df["Avg Total Purchase Price"].map("${:,.2f}".format)
purchases_group_df.head()

*** THIS IS OK *** PURCHASING ANALYSIS (GENDER)


,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase Price
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [544]:
# AGE DEMOGRAPHICS
print("*** NEED TO ADD PERCENTAGE *** AGE DEMOGRAPHICS")

a_purchases_df = purchase_file_df[["SN","Age"]].drop_duplicates()
#a_purchases_df

a_bins = [0,9,14,19,24,29,34,39,999]
a_groups =["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

a_purchases_df["Age Group"] = pd.cut(a_purchases_df["Age"], a_bins, labels=a_groups, include_lowest=True)
##a_purchases_df = a_purchases_df.sort_values(["Age Group"])
#a_purchases_df

age_group = a_purchases_df.groupby("Age Group")
age_group.head()
age_group[["Age Group"]].count()

#age_group["Percentage of Players"] = "TBD"
#age_group["Percentage of Players"] = age_group["Age"] / distinct_tot
#distinct_tot
#age_group.head()

#age_group.head()
#age_group[["Age"]].count()
#age_group.head()
#bin_counts = a_purchases_df["Age Group"].value_counts()
#bin_percents = (a_purchases_df["Age Group"].value_counts() / distinct_tot) * 100
#bin_counts.head()

##age_summary = pd.DataFrame({"Total Count": bin_counts, "Percentage of Players": bin_percents})
##age_summary["Percentage of Players"] = age_summary["Percentage of Players"].map("{:.2f}%".format)
##age_summary = age_summary.sort_values(["Age Group"])
#age_summary


*** NEED TO ADD PERCENTAGE *** AGE DEMOGRAPHICS


,Age Group
Age Group,
<10,17
10-14,22
15-19,107
20-24,258
25-29,77
30-34,52
35-39,31
40+,12


In [390]:
# PURCHASING ANALYSIS (AGE)
print("PURCHASING ANALYSIS (AGE)")

PURCHASING ANALYSIS (AGE)


In [9]:
# TOP SPENDERS
print("TOP SPENDERS")

TOP SPENDERS


In [446]:
# MOST POPULAR ITEMS
print("*** TRY TO LEFT-JUSTIFY ITEM NAME COLUMN *** MOST POPULAR ITEMS")

item_df = purchase_file_df[["Item ID", "Item Name", "Price"]]
item_df.head()

item_group_df = item_df.groupby(["Item ID","Item Name"]).count()
item_group_df.columns = ["Purchase Count"]
item_group_df["Item Price"] = item_df.groupby(["Item ID","Item Name"]).mean()

item_group_df["Total Purchase Value"] = item_group_df["Purchase Count"] * item_group_df["Item Price"]
item_group_df["Item Price"] = item_group_df["Item Price"].map("${:,.2f}".format)
item_group_df["Total Purchase Value"] = item_group_df["Total Purchase Value"].map("${:,.2f}".format)
item_group_df = item_group_df.sort_values("Purchase Count", ascending=False)
item_group_df.head().style.set_properties(**{'text-align': 'left'})



*** TRY TO LEFT-JUSTIFY ITEM NAME COLUMN *** MOST POPULAR ITEMS


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [451]:
# ***TOTAL PURCHASE VALUE IS NOT CORRECT *** MOST PROFITABLE ITEMS


item_group_df = item_group_df.sort_values("Total Purchase Value", ascending=False)
item_group_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
63,Stormfury Mace,2,$4.99,$9.98
29,"Chaos, Ender of the End",5,$1.98,$9.90
173,Stormfury Longsword,2,$4.93,$9.86
38,"The Void, Vengeance of Dark Magic",4,$2.37,$9.48
143,Frenzied Scimitar,6,$1.56,$9.36
